In [15]:
import cv2
import numpy as np

In [2]:
cfg = 'yolov4.cfg'
weight = 'yolov4.weights'
labels = 'coco.names'

In [18]:
thresh_for_detection = 0.5
nms_thresh = 0.4
inp_size = 416
cam = 0

In [21]:
net = cv2.dnn.readNet(weight, cfg)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [7]:
final_layers=net.getUnconnectedOutLayersNames()

In [9]:
final_layers

('yolo_139', 'yolo_150', 'yolo_161')

In [11]:
with open(labels, 'r') as f:
    names = [line.strip() for line in f.readlines()]

In [13]:
names

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [ ]:
cap = cv2.VideoCapture(1)
while True:
    isread, frame = cap.read()
    if not isread:
        break
    h,w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(image=frame, scalefactor=1/255.0, size=(inp_size, inp_size), swapRB=True)
    net.setInput(blob)
    out = net.forward(final_layers)
    bbox, confidence, class_id = [], [], []
    for output in out:
        for det in output:
            scores = det[5:]  #focus on indexing
            clas = np.argmax(scores)  #returns index
            conf = scores[clas]
            if conf>=thresh_for_detection and names[clas]=='person':
                centre_x = int(det[0]*w)
                centre_y = int(det[1]*h)
                width = int(det[2]*w)
                height = int(det[3]*h)
                x = int(centre_x-w/2)
                y = int(centre_y-height/2)
                bbox.append([x,y,width,height])
                class_id.append(clas)
                confidence.append(conf)
    indcs=cv2.dnn.NMSBoxes(bbox, confidence, thresh_for_detection, nms_thresh)
    if len(indcs)>0:
        indcs = indcs.flatten() if indcs.ndim > 1 else indcs
        for i in indcs:
            x,y,w,h = bbox[i]
            c = confidence[i]
            cl = class_id[i]

            cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
            text = f'{names[cl]} : {c}'
            cv2.putText(frame, text, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.2, (255,0,0), 2)
    
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()